# Preliminaries

In [ ]:
import subprocess

In [ ]:
if 'google.colab' in str(get_ipython()):
    subprocess.run('wget -q https://raw.githubusercontent.com/plausibilities/sars/develop/scripts.sh', shell=True)
    subprocess.run('chmod u+x scripts.sh', shell=True)
    subprocess.run('./scripts.sh', shell=True)

<br>

## Paths

In [ ]:
import os
import pathlib
import sys

In [ ]:
if not 'google.colab' in str(get_ipython()):
    
    notebooks = os.path.split(os.getcwd())[0]
    parent = str(pathlib.Path(notebooks))
    sys.path.append(parent)


<br>

## Libraries

In [ ]:
%matplotlib inline

In [ ]:
import arviz as az
import graphviz
import dotmap
import pymc3 as pm

import logging

import numpy as np
import pandas as pd
import scipy as sp

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import theano
import collections

<br>

**Custom**

In [ ]:
import sars.graphics.settings
import sars.graphics.relational

In [ ]:
import sars.capita.feed.data
import sars.capita.feed.segmentation

<br>

## Set-up

**Display**<br>
If the display screen supports **retina display**, a figure's resolution can be enhanced via


In [ ]:
%config InlineBackend.figure_format = 'retina'

<br>

**Jupyter Settings**

In [ ]:
settings = sars.graphics.settings.Settings()

settings.layout()
settings.aesthetics()

<br>

**Relational Graph Labels**

In [ ]:
RelationalGraphLabels = collections.namedtuple(typename='RelationalGraphLabels',
                                               field_names=['title', 'xlabel', 'ylabel'])

<br>

## Logging

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>
<br>

# Data

## Read

In [ ]:
data = sars.capita.feed.data.Data().read()
logger.info('\n{}\n'.format(data.head()))

<br>

**In Focus**

In [ ]:
stusps = 'ND'

<br>

**Hence**

In [ ]:
segmentation = sars.capita.feed.segmentation.Segmentation(data=data)

features, observations, frame = segmentation.exc(
    stusps=stusps, independent=['ndays'], dependent=['positiveRate', 'hospitalizedRate', 'deathRate'])

logger.info('The shape & type of features\n{}, {}\n'.format(features.shape, type(features)))
logger.info('The shape & type of observations\n{}, {}\n'.format(observations.shape, type(observations)))
logger.info('The frame\n{}, {}\n'.format(frame.shape, type(frame)))

In [ ]:
futures = segmentation.futures(features=features, steps=np.array([1]), ahead=25)

logger.info('The shape of futures\n{}\n'.format(futures.shape))

<br>

An illustration of the independent & dependent variables

In [ ]:
import sars.capita.models.covariates.graphs

In [ ]:
graphs = sars.capita.models.covariates.graphs.Graphs()

In [ ]:
ax = graphs.baseline(
    data=frame, labels=RelationalGraphLabels._make(['\ndata\n', '\ndays thus far', 'ln(observations\n/100K)\n'])
)
ax.legend(loc='upper left', fontsize='small');

<br>
<br>

# Inference

## Structures

Preparing data structures for the inference model

In [ ]:
import sars.capita.models.covariates.structures

In [ ]:
structures = sars.capita.models.covariates.structures.Structures()

<br>

**Scaling Objects & Structures** of the:

* Independent Variable/s

In [ ]:
XSCALER, XSCALED = structures.exc(tensor=features)

logger.info('\nThe shape & type of the scaled features: {}, {}.\n'.
       format(XSCALED.shape, type(XSCALED)))


<br>

* Dependent Variable/s

In [ ]:
YSCALER, YSCALED = structures.exc(tensor=observations)

logger.info('\nThe shape & type of the scaled observations: {}, {}.\n'.
       format(YSCALED.shape, type(YSCALED)))


<br>
<br>

## Model

**Attributes**

In [ ]:
import sars.capita.models.covariates.attributes

In [ ]:
sections_ = XSCALED.shape[0]
instances_ = np.ones(shape=XSCALED.shape[0], dtype=int)

attributes = sars.capita.models.covariates.attributes.Attributes(
    independent=XSCALED, dependent=YSCALED, sections_=sections_, instances_=instances_)

<br>

Hence

In [ ]:
measures = attributes.measures()
sections = attributes.sections()

logger.info('\n# of records: {N}, # of independent variables: {M}, # of dependent variables: {P}\n'.\
            format(N = measures.N, M = measures.M, P = measures.P))

logger.info('\n# of sections: {sections_}\n'.format(sections_ = sections.sections_))

<br>

**Inference**

In [ ]:
import sars.capita.models.covariates.inference

In [ ]:
inference = sars.capita.models.covariates.inference.Inference(
    independent=XSCALED, dependent=YSCALED, measures=measures, sections=sections)

In [ ]:
model, likelihood, trace, maximal, arviztrace = inference.exc()